## Week 4 Final Exercise: Python Docstring Generator
Parses any Python script, detects undocumented modules / classes / functions / async functions,
and adds PEP-257 docstrings — **without touching a single character of existing code**.

In [ ]:
%pip install -q gradio openai python-dotenv

In [1]:
import os, ast
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPENROUTER_URL    = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

MODELS = [
    "meta-llama/llama-3.1-8b-instruct",
    "mistralai/mistral-7b-instruct",
    "openai/gpt-4o-mini",
    "anthropic/claude-3-haiku",
    "qwen/qwen-2.5-7b-instruct",
]

DOC_STYLES = ["Google", "NumPy", "reStructuredText"]

SYSTEM_PROMPT = """You are a Python documentation expert.
Write ONLY the inner text of a PEP-257 docstring — no triple quotes, no code, no extra explanation.
Follow the requested style. Include Args / Returns / Raises sections only when relevant."""

In [3]:
DOCSTRING_TARGETS = (ast.Module, ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef)

def _needs_docstring(node):
    if not node.body:
        return False
    first = node.body[0]
    return not (isinstance(first, ast.Expr)
                and isinstance(first.value, ast.Constant)
                and isinstance(first.value.value, str))

def _indent_of(lines, node):
    line = lines[node.body[0].lineno - 1]
    return " " * (len(line) - len(line.lstrip()))

def _snippet(lines, node):
    if isinstance(node, ast.Module):
        return "\n".join(lines[:min(30, len(lines))])
    start = node.lineno - 1
    end   = min(getattr(node, "end_lineno", start + 30), start + 30)
    return "\n".join(lines[start:end])

def _kind(node):
    return {ast.Module: "module", ast.ClassDef: "class",
            ast.FunctionDef: "function", ast.AsyncFunctionDef: "async function"}[type(node)]

In [4]:
def _ask_docstring(snippet, name, kind, style, model, client):
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": f"Write a {style}-style docstring for this Python {kind} '{name}':\n\n{snippet}"},
        ],
        temperature=0.3,
    )
    return resp.choices[0].message.content.strip()

In [5]:
def _add_docstrings(source, style, model):
    client  = OpenAI(base_url=OPENROUTER_URL, api_key=OPENROUTER_API_KEY)
    lines   = source.splitlines()
    tree    = ast.parse(source)

    targets = [
        node for node in ast.walk(tree)
        if isinstance(node, DOCSTRING_TARGETS) and _needs_docstring(node)
    ]
    # reverse order: insert from bottom up so line numbers of unprocessed nodes stay valid
    targets.sort(key=lambda n: n.body[0].lineno, reverse=True)

    added = 0
    for node in targets:
        insert_at = node.body[0].lineno - 1          # 0-indexed
        indent    = _indent_of(lines, node)
        name      = getattr(node, "name", "<module>")
        doc_text  = _ask_docstring(_snippet(lines, node), name, _kind(node), style, model, client)

        doc_lines = doc_text.strip().splitlines()
        block = (
            [f'{indent}"""{doc_lines[0]}"""']
            if len(doc_lines) == 1
            else [f'{indent}"""'] + [f'{indent}{l}' for l in doc_lines] + [f'{indent}"""']
        )
        lines[insert_at:insert_at] = block
        added += 1

    return "\n".join(lines), added

In [ ]:
def run(code, file, model, style):
    if file is not None:
        code = open(file).read()
    if not (code or "").strip():
        return "", "Paste some code or upload a .py file."
    try:
        result, added = _add_docstrings(code, style, model)
        return result, f"Done — added {added} docstring(s)."
    except SyntaxError as e:
        return code, f"Syntax error in source: {e}"
    except Exception as e:
        return code, f"Error: {e}"


with gr.Blocks(title="Python Docstring Generator", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Python Docstring Generator\nAdds PEP-257 docstrings to undocumented modules, classes, and functions - zero code modification.")

    with gr.Row():
        with gr.Column(scale=1):
            model  = gr.Dropdown(choices=MODELS, value=MODELS[0], label="Model")
            style  = gr.Dropdown(choices=DOC_STYLES, value="Google", label="Docstring Style")
            upload = gr.File(label="Upload .py file", file_types=[".py"])
            btn    = gr.Button("Add Docstrings", variant="primary")
            status = gr.Textbox(label="Status", interactive=False)

        with gr.Column(scale=2):
            code_in  = gr.Code(language="python", label="Input Code")
            code_out = gr.Code(language="python", label="Output (with docstrings)", interactive=False)

    upload.change(fn=lambda f: open(f).read() if f else "", inputs=upload, outputs=code_in)
    btn.click(fn=run, inputs=[code_in, upload, model, style], outputs=[code_out, status])

demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
